In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:

pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)



Training set (100000, 28, 28) (100000,)
Validation set (5000, 28, 28) (5000,)
Test set (5000, 28, 28) (5000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 784) (100000, 10)
Validation set (5000, 784) (5000, 10)
Test set (5000, 784) (5000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


In [31]:

batch_size = 128
hidden_layer_size1 = 1024
hidden_layer_size2 = 300
beta = 0.0
dropout_probability = 0.5
initial_learning_rate = 0.001

#num_zeroes = int(hidden_layer_size/2)
#halfzeroes = np.concatenate((np.ones(hidden_layer_size - num_zeroes)*2,np.zeros(num_zeroes)))

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size1], stddev = 1.0))
  biases1 = tf.Variable(tf.zeros([hidden_layer_size1]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size1, hidden_layer_size2], stddev = 1.0))
  biases2 = tf.Variable(tf.zeros([hidden_layer_size2]))
    
  weights3 = tf.Variable(
    tf.truncated_normal([hidden_layer_size2, num_labels], stddev = 1.0))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  def forward_prop(dataset, bDropout):
    logits1 = tf.matmul(dataset, weights1) + biases1
    relu1 = tf.nn.relu(logits1)
    if bDropout:
        relu1 = tf.nn.dropout(relu1, dropout_probability)
    logits2 = tf.matmul(relu1, weights2) + biases2
    relu2 = tf.nn.relu(logits2)
    if bDropout:
        relu2 = tf.nn.dropout(relu2, dropout_probability)
    logits3 = tf.matmul(relu2, weights3) + biases3
    return logits3
    
  logits = forward_prop(tf_train_dataset, True)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+\
        beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3))
  
  # Optimizer
  #global_step = tf.Variable(0)  # count the number of steps taken.
  #learning_rate = tf.train.exponential_decay(0.05, global_step, 100000, 0.8)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  optimizer = tf.train.AdagradOptimizer(initial_learning_rate).minimize(loss)
  #optimizer = tf.train.GradientDescentOptimizer(initial_learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(forward_prop(tf_train_dataset, False))
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, False))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, False))

In [32]:
num_steps = 10001

new_train_dataset = train_dataset
new_train_labels = train_labels

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (new_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = new_train_dataset[offset:(offset + batch_size), :]
    batch_labels = new_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 8883.989258
Minibatch accuracy: 3.9%
Validation accuracy: 7.8%
Minibatch loss at step 500: 3391.974121
Minibatch accuracy: 59.4%
Validation accuracy: 51.6%
Minibatch loss at step 1000: 3098.852539
Minibatch accuracy: 60.2%
Validation accuracy: 60.4%
Minibatch loss at step 1500: 2888.838379
Minibatch accuracy: 52.3%
Validation accuracy: 65.1%
Minibatch loss at step 2000: 2463.663086
Minibatch accuracy: 72.7%
Validation accuracy: 67.6%
Minibatch loss at step 2500: 2344.093506
Minibatch accuracy: 68.0%
Validation accuracy: 69.1%
Minibatch loss at step 3000: 2473.082031
Minibatch accuracy: 71.1%
Validation accuracy: 70.2%
Minibatch loss at step 3500: 2233.912598
Minibatch accuracy: 71.1%
Validation accuracy: 71.0%
Minibatch loss at step 4000: 1763.178345
Minibatch accuracy: 74.2%
Validation accuracy: 71.6%
Minibatch loss at step 4500: 1842.671997
Minibatch accuracy: 74.2%
Validation accuracy: 72.4%
Minibatch loss at step 5000: 1669.142578
Minibatch acc